In [53]:
import subprocess
import re
from pathlib import Path
import time

In [42]:
# === Configuration ===
data_path = "/xustrg0/2024B8049"
run_number = "222753"
process_path = "/UserData/fperakis/test_data_2025/"
mask_path = process_path+"utilities/empty_mask.npy"
output_path = process_path+"processed"
poni_file = process_path+"utilities/geometry_test.poni"
nbins = 150
n_phi = 72
n_chunks = 10  # total number of chunks

In [44]:
# === Submit array job ===
env_vars = ",".join([
    f"DATA_PATH={data_path}",
    f"RUN_NUMBER={run_number}",
    f"MASK_PATH={mask_path}",
    f"OUTPUT_PATH={output_path}",
    f"PONI_FILE={poni_file}",
    f"NBINS={nbins}",
    f"N_PHI={n_phi}",
    f"N_CHUNKS={n_chunks}"
])

submit_cmd = [
    "qsub", "-J", f"0-{n_chunks-1}",
    "-v", env_vars,
    "submit_chunk.pbs"
]

array_result = subprocess.run(submit_cmd, capture_output=True, text=True)

# === Extract array job ID ===
match = re.search(r"(\d+)", array_result.stdout)
if not match:
    raise RuntimeError("Failed to parse array job ID. Output:\n" + array_result.stdout)

array_job_id = match.group(1)
print(f"\nArray job submitted with ID: {array_job_id}")




Array job submitted with ID: 8421956


In [54]:
# === Check job status ===
!qstat -u fperakis

In [55]:
!ls -l /UserData/fperakis/test_data_2025/processed/

total 865792
-rw-r--r-- 1 fperakis nilsson  51843824 May 15 22:46 Iq_222753_chunk00.h5
-rw-r--r-- 1 fperakis nilsson  51843824 May 15 22:46 Iq_222753_chunk01.h5
-rw-r--r-- 1 fperakis nilsson  51843824 May 15 22:46 Iq_222753_chunk02.h5
-rw-r--r-- 1 fperakis nilsson  51843824 May 15 22:46 Iq_222753_chunk03.h5
-rw-r--r-- 1 fperakis nilsson  51843824 May 15 22:46 Iq_222753_chunk04.h5
-rw-r--r-- 1 fperakis nilsson  51843824 May 15 22:46 Iq_222753_chunk05.h5
-rw-r--r-- 1 fperakis nilsson  51843824 May 15 22:46 Iq_222753_chunk06.h5
-rw-r--r-- 1 fperakis nilsson  51843824 May 15 22:46 Iq_222753_chunk07.h5
-rw-r--r-- 1 fperakis nilsson  51843824 May 15 22:46 Iq_222753_chunk08.h5
-rw-r--r-- 1 fperakis nilsson  51757424 May 15 22:46 Iq_222753_chunk09.h5
-rw-r--r-- 1 fperakis nilsson 518317424 May 15 22:29 Iq_222753_combined.h5


In [56]:
# === Submit dependent merge job ===

# Define parameters
output_file = Path(output_path) / f"Iq_{run_number}_combined.h5"

# Submit the PBS job
merge_cmd = [
    "qsub",
    #"-W", f"depend=afterokarray:{array_job_id}",
    "-v", f"INPUT_DIR={Path(output_path)},RUN_NUMBER={run_number},OUTPUT_FILE={output_file}",
    "combine_chunks.pbs"
]

result = subprocess.run(merge_cmd, capture_output=True, text=True)

# Output results
if result.returncode == 0:
    merge_job_id = result.stdout.strip()
    print(f"Merge job submitted successfully with job ID: {merge_job_id}")
else:
    print("Failed to submit merge job.")
    print(result.stderr)

Merge job submitted successfully with job ID: 8421960.fep01


In [58]:
!ls -l /UserData/fperakis/test_data_2025/processed/

total 145920
-rw-r--r-- 1 fperakis nilsson 518317424 May 15 22:47 Iq_222753_combined.h5


In [59]:
array_job_id

'8421956'